# Results Evaluation

In [1]:
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_colwidth', 400)

import matplotlib.pyplot as plt
import seaborn as sns

import _pickle as pickle

from sklearn.metrics import accuracy_score

from tools.dataset_tools import Dataset


In [2]:
results_folder = os.path.expanduser('~') + f'/proj/XKE_results/'
datasets_list = ['NELL186', 'FB15K237']

In [3]:
df = pd.DataFrame(columns = ['Dataset', 'Embedding', 'Timestamp_EMB', 'Timestamp_SFE', 'Timestamp_XKE'])
results_compilation = pd.read_csv('metrics_template.tsv', sep='\t', index_col=0)

for dataset in datasets_list:
    folder = results_folder + f'{dataset}/xke_explain/'
    timestamps = os.listdir(folder)
    for timestamp in timestamps:
        split_timestamp = timestamp.split('_')
        xke_timestamps = os.listdir(folder + f'{timestamp}/')
        for xke_timestamp in xke_timestamps:
            df.loc[len(df)] = [dataset, split_timestamp[0], split_timestamp[1], split_timestamp[2][3:], xke_timestamp]
            try:
                overall_results = pd.read_csv(f'{folder}{timestamp}/{xke_timestamp}/overall_metrics.tsv', sep='\t', index_col=0)
            except:
                print(f'Skipping xke_timestamp {xke_timestamp}, could no find overral_metrics file!')
                continue
            # if len(results_compilation) == 0:
            #     results_compilation = overall_results
            # else:
            results_compilation = results_compilation.join(overall_results, how='left')
            # results_compilation = pd.merge(results_compilation, overall_results, left_index=True, right_index=True)

# results_compilation = results_compilation.sort_values(by='idx').T
results_compilation = results_compilation.sort_values(by='idx').drop(columns=['metric_type', 'idx']).T


# print('finished loading!')

# results_compilation['XKEe:test_predictions_corrected'] = results_compilation['XKEe:false_positives_corrected'].astype(int) + results_compilation['XKEe:false_negatives_corrected'].astype(int)


Skipping xke_timestamp 2403091230, could no find overral_metrics file!
Skipping xke_timestamp 2403091235, could no find overral_metrics file!


In [4]:
# Some post-processing of the columns to properly format the tables

percentages = [
    'emb:overall_test_acc',

    'pru:test_triples_w_feat', 
    
    'sfe:test_triples_w_feat', 
    'sfe:test_triples_w_active_feat', 
    'sfe:test_acc (micro_avg)',
    'sfe:test_f1 (micro_avg)', 
    'sfe:overall_interpretability_index',
    'sfe:overall_interpretability_index(triples_with_feature)',
    
    'xke:test_triples_w_active_feat', 
    'xke:fidelity_test_f1 (micro_avg)',
    'xke:fidelity_test_acc (micro_avg)', 
    'xke:overall_interpretability_index',
    'xke:overall_interpretability_index(triples_with_feature)',
    

    'XKEe:test_triples_w_active_feat', 
    'XKEe:fidelity_test_acc (micro_avg)', 
    'XKEe:fidelity_test_f1 (micro_avg)',
    'XKEe:overall_interpretability_index',
    'XKEe:overall_interpretability_index(triples_with_feature)'
    ]
for col in percentages:
    results_compilation[col] = results_compilation[col].astype(float)*100

textual = ['pru:top_pop', 'pru:top_avg_rel_sim']
for col in textual:
    results_compilation[col] = results_compilation[col].apply(lambda x: x[:4])

to_float = ['sfe:features_triples_ratio', 'xke:test_avg_feat_per_triple_w_active_feat', 'XKEe:test_avg_feat_per_triple_w_active_feat']
for col in to_float:
    results_compilation[col] = results_compilation[col].astype(float)

In [5]:
results_compilation

,dat:dataset,dat:total_rels,dat:tested_rels,dat:train_triples_neg,dat:train_triples_pos,dat:train_triples_total,dat:test_triples_neg,dat:test_triples_pos,dat:test_triples_total,emb:model,emb:timestamp,emb:overall_test_acc,emb:rel_train_tn,emb:rel_train_fp,emb:rel_train_fn,emb:rel_train_tp,emb:rel_train_f1,emb:rel_train_acc,emb:rel_test_tn,emb:rel_test_fp,emb:rel_test_fn,emb:rel_test_tp,emb:rel_test_f1,emb:rel_test_acc,sfe:timestamp,sfe:sim_model,sfe:extraction_method,sfe:node_threshold,sfe:rel_threshold,sfe:node_relv_in,sfe:top_rels_reduced_graph,sfe:train_triples_neg_w_feat,sfe:train_triples_pos_w_feat,sfe:test_triples_neg_w_feat,sfe:test_triples_pos_w_feat,sfe:train_triples_w_feat,sfe:test_triples_w_feat,sfe:train_feature_count,sfe:test_feature_count,sfe:common_feature_count,sfe:features_triples_ratio,sfe:feature_extraction_elapsed_time,sfe:feature_vectorization_elapsed_time,sfe:feature_compilation_elapsed_time,sfe:feature_compilation2_elapsed_time,sfe:feature_dataframe_elapsed_time,sfe:feature_sim_calculation_elapsed_time,sfe:feature_storing_elapsed_time,sfe:elapsed_time,pru:prunning,pru:max_l,pru:min_rel_sim,pru:node_relv_in,pru:top_pop,pru:top_pos,pru:top_neg,pru:top_avg_rel_sim,pru:train_triples_neg_w_feat,pru:train_triples_pos_w_feat,pru:test_triples_neg_w_feat,pru:test_triples_pos_w_feat,pru:train_triples_w_feat,pru:test_triples_w_feat,pru:train_feature_count,pru:test_feature_count,pru:common_feature_count,sfe:train_tn,sfe:train_fp,sfe:train_fn,sfe:train_tp,sfe:train_acc (micro_avg),sfe:train_f1 (micro_avg),sfe:test_AP,sfe:test_tn,sfe:test_fp,sfe:test_fn,sfe:test_tp,sfe:test_acc (micro_avg),sfe:test_f1 (micro_avg),sfe:train_triples_neg_w_active_feat,sfe:train_triples_pos_w_active_feat,sfe:test_triples_neg_w_active_feat,sfe:test_triples_pos_w_active_feat,sfe:train_triples_w_active_feat,sfe:test_triples_w_active_feat,sfe:train_pos_avg_feat_per_triple_w_active_feat,sfe:train_neg_avg_feat_per_triple_w_active_feat,sfe:test_pos_avg_feat_per_triple_w_active_feat,sfe:test_neg_avg_feat_per_triple_w_active_feat,sfe:train_avg_feat_per_triple_w_active_feat,sfe:test_avg_feat_per_triple_w_active_feat,sfe:neg_active_feat,sfe:pos_active_feat,sfe:active_feat,sfe:active_rels,sfe:overall_interpretability_index,sfe:overall_interpretability_index(triples_with_feature),sfe:reg_elapsed_time,xke:fidelity_train_tn,xke:fidelity_train_fp,xke:fidelity_train_fn,xke:fidelity_train_tp,xke:fidelity_train_acc (micro_avg),xke:fidelity_train_f1 (micro_avg),xke:fidelity_test_AP,xke:fidelity_test_tn,xke:fidelity_test_fp,xke:fidelity_test_fn,xke:fidelity_test_tp,xke:fidelity_test_acc (micro_avg),xke:fidelity_test_f1 (micro_avg),xke:accuracy_train_tn,xke:accuracy_train_fp,xke:accuracy_train_fn,xke:accuracy_train_tp,xke:accuracy_train_acc (micro_avg),xke:accuracy_train_f1 (micro_avg),xke:accuracy_test_AP,xke:accuracy_test_tn,xke:accuracy_test_fp,xke:accuracy_test_fn,xke:accuracy_test_tp,xke:accuracy_test_acc (micro_avg),xke:accuracy_test_f1 (micro_avg),xke:benchmark_model,xke:benchmark_fidelity,xke:benchmark_f1_fidelity,xke:train_triples_neg_w_active_feat,xke:train_triples_pos_w_active_feat,xke:test_triples_neg_w_active_feat,xke:test_triples_pos_w_active_feat,xke:train_triples_w_active_feat,xke:test_triples_w_active_feat,xke:reg_elapsed_time,xke:train_pos_avg_feat_per_triple_w_active_feat,xke:train_neg_avg_feat_per_triple_w_active_feat,xke:test_pos_avg_feat_per_triple_w_active_feat,xke:test_neg_avg_feat_per_triple_w_active_feat,xke:train_avg_feat_per_triple_w_active_feat,xke:test_avg_feat_per_triple_w_active_feat,xke:neg_active_feat,xke:pos_active_feat,xke:active_feat,xke:active_rels,xke:active_feat_count_p_len_1,xke:active_feat_count_p_len_2,xke:active_feat_count_p_len_3,xke:active_feat_count_p_len_4,xke:overall_interpretability_index,xke:overall_interpretability_index(triples_with_feature),XKEe:false_negatives_corrected,XKEe:false_positives_corrected,XKEe:fidelity_test_tn,XKEe:fidelity_test_fp,XKEe:fidelity_test_fn,XKEe:fidelity_test_tp,XKEe:fidelity_test_

In [6]:
analogy_timestamps = [

    '2108251740', # NELL186 pru:top_avg_rel_sim = 0.2
    '2108251747', # NELL186 pro:top_pop = 0.2
    # '2109012224', # NELL186 node_relv_in pru:top_avg_rel_sim = 0.2
    '2108251753', # NELL186 no prunning

    '2108260741', # FB15K237 pru:top_avg_rel_sim = 0.2
    '2108260952' # FB15K237 pru:top_pop = 0.2
    # '2109020616'  # FB15K237 Analogy node_relv_in pru:top_avg_rel_sim = 0.2

]


transe_timestamps = [

    '2108251555',  # NELL186 pru:top_avg_rel_sim = 0.2
    '2108251605',  # NELL186 pru:top_pop = 0.2
    # '2109012040',  # NELL186 node_relv_in pru:top_avg_rel_sim = 0.2
    '2108251618',  # NELL186 pru: none


    '2108252017',  # FB15K237 pru:top_avg_rel_sim = 0.2
    '2108252351'#,  # FB15K237 pru:top_pop = 0.2
    # '2109020353'   # FB15K237 node_relv_in pru:top_avg_rel_sim = 0.2 
]

all_timestamps = transe_timestamps + analogy_timestamps

timestamp_dict = {
    #Analogy
    '2108251740' : 'top_pc_rels:0.2', # NELL186 pru:top_avg_rel_sim = 0.2
    '2108251747' : 'top_pop:0.2', # NELL186 pro:top_pop = 0.2
    # '2109012224' : 'node_context search / top_pc_rels:0.2' , # NELL186 node_relv_in pru:top_avg_rel_sim = 0.2
    '2108251753' : 'no_feature_selection', # NELL186 no prunning

    '2108260741' : 'top_pc_rels:0.2',  # FB15K237 pru:top_avg_rel_sim = 0.2
    '2108260952' : 'top_pop:0.2', # FB15K237 pru:top_pop = 0.2
    # '2109020616' : 'node_context search / top_pc_rels:0.2', # FB15K237 Analogy node_relv_in pru:top_avg_rel_sim = 0.2

    #TransE
    '2108251555' : 'top_pc_rels:0.2', # NELL186 pru:top_avg_rel_sim = 0.2
    '2108251605' : 'top_pop:0.2', # NELL186 pru:top_pop = 0.2
    # '2109012040' : 'node_context search / top_pc_rels:0.2',  # NELL186 node_relv_in pru:top_avg_rel_sim = 0.2
    '2108251618' : 'no_feature_selection', # NELL186 pru: none

    '2108252017' : 'top_pc_rels:0.2',  # FB15K237 pru:top_avg_rel_sim = 0.2
    '2108252351' : 'top_pop:0.2', # FB15K237 pru:top_pop = 0.2 
    # '2109020353' : 'node_context search / top_pc_rels:0.2'  # FB15K237 node_relv_in pru:top_avg_rel_sim = 0.2
}

In [7]:
complete_table = results_compilation.T[all_timestamps]
complete_table

,2108251555,2108251605,2108251618,2108252017,2108252351,2108251740,2108251747,2108251753,2108260741,2108260952
dat:dataset,NELL186,NELL186,NELL186,FB15K237,FB15K237,NELL186,NELL186,NELL186,FB15K237,FB15K237
dat:total_rels,186,186,186,237,237,186,186,186,237,237
dat:tested_rels,170,170,170,217,217,171,171,171,219,219
dat:train_triples_neg,65951,65951,65951,529704,529704,66019,66019,66019,530970,530970
dat:train_triples_pos,35829,35829,35829,284330,284330,35865,35865,35865,284970,284970
dat:train_triples_total,101780,101780,101780,814034,814034,101884,101884,101884,815940,815940
dat:test_triples_neg,4978,4978,4978,20215,20215,4984,4984,4984,20231,20231
dat:test_triples_pos,4978,4978,4978,20436,20436,4984,4984,4984,20452,20452
dat:test_triples_total,9956,9956,9956,40651,40651,9968,9968,9968,40683,40683
emb:model,TransE,TransE,TransE,TransE,TransE,Analogy,Analogy,Analogy,Analogy,Analogy


In [9]:
complete_table.to_csv('Thesis Experiment Results.csv')